In [ ]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

# Trade Directionality Hedging

### Add a short historical data sample for covariance

In [ ]:
portfolio = Portfolio()
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
Q = historical_chgs.cov().to_numpy()
Q

### PCA Approach

In [ ]:
lambd, E = portfolio.pca(Q)
S_ini = np.array([-1, 2.0, -1, 0])[:, np.newaxis]
portfolio.pca_hedge_adjustment(None, Q, S_ini)

The problem with the above minimal adjustment is that it includes a 30Y bucket even though 30Y was not considered in the original trade. We can explicitly set that bucket to be zero in our directionality adjustment.

In [ ]:
x = portfolio.pca_hedge_adjustment(None, Q, S_ini, L=[3])
x

In [ ]:
S = (S_ini + x)
S

In [ ]:
delta_r = np.array([17.2, 18.1, 12.5, 8.1])[:, np.newaxis]
print("PnL: ", np.matmul(S.T, delta_r))

Below we can calculate the non centralised PC multipliers and explain the PnL from PCA risks and multipliers. The PnL value is invariant.

In [ ]:
non_cent_multipliers = np.matmul(delta_r.T, E).T
pca_risk = np.matmul(E.T, S)
print(non_cent_multipliers, "\n", pca_risk)

In [ ]:
print("PnL from PCA: ", np.matmul(pca_risk.T, non_cent_multipliers))

Since the risk to PC1 is zero we can determine the residuals if we exclude every PC except the first

In [ ]:
nc_multiplier_neg = non_cent_multipliers.copy()
nc_multiplier_neg[1,0] = 0
nc_multiplier_neg[2,0] = 0
nc_multiplier_neg[3,0] = 0

In [ ]:
delta_r_adj = np.matmul(E, nc_multiplier_neg)
delta_r_adj

In [ ]:
residuals = delta_r - delta_r_adj
residuals

The PnL from residuals is invariant since PC1 risk was explcitly set to be zero.

In [ ]:
print("PnL from residuals: ", np.matmul(S.T, residuals))

### CoVaR Minimisation Approach

In [ ]:
S_ini1 = np.array([-1, 2, -1.0, 0])[:, np.newaxis]
S_ini2 = np.array([-1, 2, -1.0, 0])[:, np.newaxis]

In [ ]:
x1 = portfolio.covar_mmt(None, Q, [1], S_ini1)
x2 = portfolio.covar_mmt(None, Q, [0,2], S_ini2)

In [ ]:
S1 = (S_ini1 + x1) 
S1 * 1.9394 / S1[1,0]

In [ ]:
S2 = (S_ini2 + x2)
S2 * 1.9394 / S2[1,0]

### Multivariate Least Squares Regression

In [ ]:
delta_r = historical_chgs.iloc[:9,:].to_numpy()
delta_r = delta_r - delta_r.mean(axis=0)
delta_r

Assume a 2s5s10s trade hedged for 5Y only

In [ ]:
S_ini = np.array([-1, 2.0, -1, 0])[:, np.newaxis]
y = np.matmul(delta_r, S_ini)
x_1 = np.array([0, 1.0, 0, 0])[:, np.newaxis]
X = np.matmul(delta_r, x_1)

In [ ]:
beta = np.matmul(1/np.matmul(X.T, X), np.matmul(X.T, y))

In [ ]:
S_trade = -np.matmul(x_1, beta)
S_trade

In [ ]:
S = S_ini + S_trade
S * 1.9394 / S[1,0]

This result is the same as the CoVaR minimisation against the single 5Y instrument

Now assume a hedge to 2y and 10y.

In [ ]:
x_j = np.array([
    [1.0, 0],
    [0, 0],
    [0, 1.0],
    [0, 0]
])
X = np.matmul(delta_r, x_j)
beta = np.linalg.solve(np.matmul(X.T, X), np.matmul(X.T, y))
S_trade = - np.matmul(x_j, beta)
S = S_ini + S_trade
S * 1.9394 / S[1,0]

This result is the same as the multi-instrument VaR minimisation strategy measured over 2Y and 10Y.

Now we will try to hedge against the first PC

In [ ]:
x_1 = E[:, [0]]
x_1[3, 0] = 0
X = np.matmul(delta_r, x_1)
beta = np.linalg.solve(np.matmul(X.T, X), np.matmul(X.T, y))
S_trade = - np.matmul(x_1, beta)
S = S_ini + S_trade
S
# S * 1.93941304 / S[1,0]

In [ ]:
pca_risk = np.matmul(E.T, S)
pca_risk

This result is very close to the PCA minimisation approach and we see above that risk to PC1 is essentially fully hedged.